In [ ]:
#%load_ext autoreload
#%autoreload 2
%matplotlib inline
%cd '../HOTS'

In [ ]:
import os
import pickle
import datetime
import time
from mix_Network import *

dataset = 'nmnist'
records_path = '../Records/'
timestr = datetime.datetime.now().date().isoformat()
#timestr = '2020-11-25'
algo = 'lagorce'

ds = 10
#ds = 75

NbTrainingData = 7500//ds
NbTestingData = 2500//ds

tau = 5

In [ ]:
def get_nmnist(NbTrainingData, NbTestingData):
    class eventV(object):
            def __init__(self, nbevt):
                self.time = np.zeros((nbevt))
                self.address = np.zeros((nbevt, 2))
                self.polarity = np.zeros((nbevt))
                self.ImageSize = [34,34]
                self.ListPolarities = [0,1]
                
    learningset = tonic.datasets.NMNIST(save_to='../Data/',
                                train=False,
                                transform=None)
    loader = tonic.datasets.DataLoader(learningset, shuffle=True)
    
    eventslist = [next(iter(loader)) for i in range(NbTrainingData)]
    nbtot = 0
    for i in range(NbTrainingData):
        nbtot+=eventslist[i][0].shape[1]
    events_train = eventV(nbtot)
    labels_train = []
    idx = 0
    for i in range(NbTrainingData):
        for j in range(eventslist[i][0].shape[1]):
            events_train.address[idx,0] = eventslist[i][0][0,j,0].item()
            events_train.address[idx,1] = eventslist[i][0][0,j,1].item()
            events_train.time[idx] = eventslist[i][0][0,j,2].item()*1e-6
            events_train.polarity[idx] = eventslist[i][0][0,j,3].item()
            labels_train.append(eventslist[i][1])
            idx+=1
            
    eventslist = [next(iter(loader)) for i in range(NbTestingData)]
    nbtot = 0
    for i in range(NbTestingData):
        nbtot+=eventslist[i][0].shape[1]
    events_test = eventV(nbtot)
    labels_test = []
    idx = 0
    for i in range(NbTestingData):
        for j in range(eventslist[i][0].shape[1]):
            events_test.address[idx,0] = eventslist[i][0][0,j,0].item()
            events_test.address[idx,1] = eventslist[i][0][0,j,1].item()
            events_test.time[idx] = eventslist[i][0][0,j,2].item()*1e-6
            events_test.polarity[idx] = eventslist[i][0][0,j,3].item()
            labels_test.append(eventslist[i][1])
            idx+=1
    events_cluster = []
    return [events_train, events_test, events_cluster, labels_train, labels_test]

# ### Building matrix for logistic regression
def gather_data(events_in, labels_in,
                tau_cla=.150, # characteristic time of a digit
                sample_events=50, sample_space = 1,
                verbose=False, debug=False):

    n_events = events_in.time.shape[0]

    c_int = lambda n, d : ((n - 1) // d) + 1
    data = np.zeros((c_int(events_in.ImageSize[0], sample_space),
                     c_int(events_in.ImageSize[1], sample_space),
                     len(events_in.ListPolarities))) #tmp data

    X = np.zeros((c_int(n_events, sample_events), len(data.ravel())))
    y = np.zeros((c_int(n_events, sample_events), ))

    for i_event in range(1, n_events):
        if events_in.time[i_event]<events_in.time[i_event-1]:
            data = np.zeros((c_int(events_in.ImageSize[0], sample_space),
                     c_int(events_in.ImageSize[1], sample_space),
                     len(events_in.ListPolarities))) #tmp data 


        data *= np.exp(-(events_in.time[i_event]-events_in.time[i_event-1])/tau_cla)
        x_pos = events_in.address[i_event, 0]//sample_space
        y_pos = events_in.address[i_event, 1]//sample_space
        p = events_in.polarity[i_event]
        data[int(x_pos), int(y_pos), int(p)] = 1.

        if i_event % sample_events == sample_events//2 :
            if debug:
                print(f'DEBUG {i_event} {i_event//sample_events} ')
                print(f'DEBUG {y[i_event//sample_events]}   ')
                print(f'DEBUG  {labels_in[i_event]} ')
            X[i_event//sample_events, :] = data.ravel()
            y[i_event//sample_events] = labels_in[i_event]
            
    if verbose: print('Number of events: ' + str(X.shape[0])+' - Number of features: ' + str(X.shape[1]))

    return X, y

In [ ]:
homeo = False

fname_ = f'{records_path}/EXP_03_NMNIST/{timestr}_hots_{tau}_ms_{NbTrainingData}_{NbTestingData}'
label = '_homeo' if homeo else ''
fname_model = fname_ + '_model_' + algo + label + '.pkl'
fname_event0_o = fname_ + '_event_out_' + algo + label + '.pkl'

hots = network(krnlinit='first', tau=tau, homeo=homeo)
if not os.path.isfile(fname_model):
    loader = hots.learning1by1(dataset=dataset)
    with open(fname_model, 'wb') as file:
        pickle.dump([hots, loader], file, pickle.HIGHEST_PROTOCOL)
else:
    with open(fname_model, 'rb') as file:
        hots, loader = pickle.load(file)
if not os.path.isfile(fname_event0_o):
    _ , loader, out_train = hots.training(loader, LR=True, nb_digit = NbTrainingData)
    _ , loader, out_test = hots.training(loader, LR=True, nb_digit = NbTestingData)
    with open(fname_event0_o, 'wb') as file:
        pickle.dump([out_train, out_test], file, pickle.HIGHEST_PROTOCOL)
else:
    with open(fname_event0_o, 'rb') as file:
        out_train, out_test = pickle.load(file)

In [ ]:
homeo = True

fname_ = f'{records_path}/EXP_03_NMNIST/{timestr}_hots_{tau}_ms_{NbTrainingData}_{NbTestingData}'
label = '_homeo' if homeo else ''
fname_model = fname_ + '_model_' + algo + label + '.pkl'
fname_event0_o = fname_ + '_event_out_' + algo + label + '.pkl'

hots = network(krnlinit='first', tau=tau, homeo=homeo)
if not os.path.isfile(fname_model):
    loader = hots.learning1by1(dataset=dataset)
    with open(fname_model, 'wb') as file:
        pickle.dump([hots, loader], file, pickle.HIGHEST_PROTOCOL)
else:
    with open(fname_model, 'rb') as file:
        hots, loader = pickle.load(file)
if not os.path.isfile(fname_event0_o):
    _ , loader, out_train_hom = hots.training(loader, LR=True, nb_digit = NbTrainingData)
    _ , loader, out_test_hom = hots.training(loader, LR=True, nb_digit = NbTestingData)
    with open(fname_event0_o, 'wb') as file:
        pickle.dump([out_train_hom, out_test_hom], file, pickle.HIGHEST_PROTOCOL)
else:
    with open(fname_event0_o, 'rb') as file:
        out_train_hom, out_test_hom = pickle.load(file)

In [ ]:
verbose=False

# Performing logistic regression

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegressionCV.html


In [ ]:
# from sklearn.linear_model import LogisticRegression as LR
from sklearn.linear_model import LogisticRegressionCV as LR
# 
opts_LR = dict(max_iter=2000, # random_state=0,
               n_jobs=-1, class_weight='balanced')
#opts_LR['Cs'] = 5
opts_LR['Cs'] = 32
# TODO for a publication use 100 from 10^{-10} to 10

from sklearn.model_selection import train_test_split

def tic():
    global ttic
    ttic = time.time()
def toc():
    print(f'Done in {time.time() - ttic:.3f} s')

### Performing logistic regression on raw input spikes

In [ ]:
tic()
dataset = get_nmnist(NbTrainingData, NbTestingData)
events_train, events_test, events_cluster, labels_train, labels_test = dataset
toc()

In [ ]:
tic()
X_train, y_train = gather_data(events_train, labels_train, verbose=verbose)
toc()

In [ ]:
tic()
lr = LR(**opts_LR).fit(X_train, y_train)
print(f'Classification score for raw input is {lr.score(X_train, y_train):.3f} (train)')
toc()

In [ ]:
tic()
X_test, y_test = gather_data(events_test, labels_test, verbose=verbose)
print(f'Classification score for raw input is {lr.score(X_test, y_test):.3f} (test)')
toc()

### Performing logistic regression with or without homeostasis

In [ ]:
for homeo in [True, False]:
    print(40*'-')
    print(f'homeo={homeo}')
    print(40*'-')
    if homeo:
        events_train_o, events_test_o, labels_train, labels_test = out_train_hom[0], out_test_hom[0], out_train_hom[1], out_test_hom[1]
    else: 
        events_train_o, events_test_o, labels_train, labels_test = out_train[0], out_test[0], out_train[1], out_test[1]
        
    X_train, y_train = gather_data(events_train_o, labels_train, verbose=verbose)
    lr = LR(**opts_LR, verbose=verbose).fit(X_train, y_train)
    print(f'Classification score for homeo = {homeo} is {lr.score(X_train, y_train):.3f} (train)')
    
    X_test, y_test = gather_data(events_test_o, labels_test, verbose=verbose)
    print(f'Classification score for homeo = {homeo} is {lr.score(X_test, y_test):.3f} (test)')

### Performing logistic regression with different solvers

In [ ]:
for homeo in [False, True]:
    print(40*'-')
    print(f'{homeo=}')
    print(40*'-')
    for solver, penalty in zip(['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], ['l2', 'l2', 'l1', 'l2', 'l1']):
        events_train_o, events_test_o, labels_train, labels_test = get_events(timestr, tau=tau, homeo=homeo, verbose=verbose)

        tic = time.time()
        X_train, y_train = gather_data(events_train_o, labels_train, verbose=verbose)
        lr = LR(solver=solver, penalty=penalty, verbose=verbose, **opts_LR).fit(X_train, y_train)
        print(f'Classification score for {homeo=} is {lr.score(X_train, y_train):.3f} (train)')
        print(f'Fit done in {time.time() -tic:.3f} seconds for {solver=}')
        X_test, y_test = gather_data(events_test_o, labels_test, verbose=verbose)
        print(f'Classification score for {homeo=} / {solver=} is {lr.score(X_test, y_test):.3f} (test)')

In [ ]:
for homeo in [False, True]:
    print(40*'-')
    print(f'{homeo=}')
    print(40*'-')
    events_train_o, events_test_o, labels_train, labels_test = get_events(timestr, tau=tau, homeo=homeo, verbose=verbose)

    for tau_cla in .15 * np.logspace(-1, 1, 7, base=4):

        X_train, y_train = gather_data(events_train_o, labels_train, tau_cla=tau_cla, verbose=verbose)
        lr = LR(**opts_LR, verbose=verbose).fit(X_train, y_train)
        print(f'Classification score for {tau_cla=:.3f} is {lr.score(X_train, y_train):.3f} (train)')
        X_test, y_test = gather_data(events_test_o, labels_test, tau_cla=tau_cla, verbose=verbose)
        print(f'Classification score for {tau_cla=:.3f} is {lr.score(X_test, y_test):.3f} (test)')

In [ ]:
for homeo in [False, True]:
    print(40*'-')
    print(f'{homeo=}')
    print(40*'-')
    events_train_o, events_test_o, labels_train, labels_test = get_events(timestr, tau=tau, homeo=homeo, verbose=verbose)

    for sample_space in [4, 2, 1]:
        X_train, y_train = gather_data(events_train_o, labels_train, sample_space=sample_space, verbose=verbose)
        lr = LR(**opts_LR, verbose=verbose).fit(X_train, y_train)
        print(f'Classification score for {sample_space=} is {lr.score(X_train, y_train):.3f} (train)')
        X_test, y_test = gather_data(events_test_o, labels_test, sample_space=sample_space, verbose=verbose)
        print(f'Classification score for {sample_space=} is {lr.score(X_test, y_test):.3f} (test)')


In [ ]:
tauz = np.array([1e-5, 1e-4, 2e-4 ,3e-4, 4e-4, 5e-4, 6e-4, 7e-4,
                 8e-4, 9e-4, 1e-3, 2e-3, 2.5e-3, 3e-3, 4e-3, 5e-3, 
                 1e-2, 2e-2])
for homeo in [False, True]:
    print(40*'-')
    print(f'{homeo=}')
    print(40*'-')
    for tau_ in tauz:
        events_train_o, events_test_o, labels_train, labels_test = get_events(timestr, tau=tau_, homeo=homeo, verbose=verbose)
        
        X_train, y_train = gather_data(events_train_o, labels_train, verbose=verbose)
        lr = LR(**opts_LR, verbose=verbose).fit(X_train, y_train)
        print(f'Classification score for {homeo=} & {tau_=:.6f} is {lr.score(X_train, y_train):.3f} (train)')
        X_test, y_test = gather_data(events_test_o, labels_test, verbose=verbose)
        print(f'Classification score for {homeo=} & {tau_=:.6f} is {lr.score(X_test, y_test):.3f} (test)')
